# Reto: Oprtimizar la ubicación de productos

## Equipo - 6: The Three Engineers

| Nombre                        | Matrícula |
| :---------------------------- | :-------: |
| Rubén Pablo Barraza Maldonado | A01570838 |
| Rodrigo Zertuche López        | A00829401 |
| Arturo Garza Campuzano        | A00828096 |

## 1. Introducción

La colocación estratégica de productos en una tienda es crucial para maximizar la probabilidad de venta de productos relacionados. La problemática radica en asignar de manera óptima los productos a los pasillos. Por ello, se tiene la necesidad de utilizar técnicas de aprendizaje automático para analizar patrones de compra y optimizar la colocación de productos.

## 2. Importación y carga de datos

In [1]:
# Importar librerias
import math
import numpy as np
import pandas as pd
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from matplotlib.ticker import ScalarFormatter

# Lectura de datos
aisles_df = pd.read_csv('Data/aisles.csv')
departments_df = pd.read_csv('Data/departments.csv')
orders_df = pd.read_csv('Data/orders.csv')
order_products_prior_df = pd.read_csv('Data/order_products__prior.csv')
order_products_train_df = pd.read_csv('Data/order_products__train.csv')
products_df = pd.read_csv('Data/products.csv')

Los diferentes archivos que conforman el conjunto de datos fueron fusionados en un solo dataframe, con el fin de facilitar el análisis y visualización de todo el conjunto de datos.

In [2]:
# Agrupa los datos de las tablas en un solo data frame
order_products = pd.DataFrame()
order_products = pd.concat([order_products_prior_df, order_products_train_df])
order_products = order_products.merge(products_df, on = 'product_id', how = 'left')
order_products = order_products.merge(aisles_df, on = 'aisle_id', how = 'left')
order_products = order_products.merge(departments_df, on = 'department_id', how = 'left')
order_products = order_products.merge(orders_df, on = 'order_id', how = 'left')
order_products.head()

## 3. Relación entre productos

### 3.1 K-Means Clustering

Se calcula la cantidad de veces que cada producto ha sido ordenado y se eliminan aquellos proudctos que hayan sido ordenados muy pocas veces; según el rango intercuartílico de la variable.

In [4]:
# Contar el numero de veces que cada producto ha sido ordenado
product_order_counts = order_products.groupby('product_id').size().reset_index(name='product_count')

# Mostrar estadisticos descriptivos
product_order_counts['product_count'].describe()

count     49685.000000
mean        680.670343
std        4987.769425
min           1.000000
25%          18.000000
50%          63.000000
75%         272.000000
max      491291.000000
Name: product_count, dtype: float64

In [5]:
# Calcular cuartiles
Q1 = product_order_counts['product_count'].quantile(0.25)
Q3 = product_order_counts['product_count'].quantile(0.75)

# Calcular el rango intercuartílico (IQR)
IQR = Q3 - Q1

# Definir los limites para detectar los outliers
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Filtrar los datos para eliminar los outliers
product_order_counts_filtered = product_order_counts[(product_order_counts['product_count'] >= lower_limit) & (product_order_counts['product_count'] <= upper_limit)]

# Mostrar estadisticos descriptivos
product_order_counts_filtered['product_count'].describe()

42575


Se calcula la cantidad de productos comprados por cada usuario y se eliminan aquellos usuarios que hayan comprado muy pocos productos; según el rango intercuartílico de la variable.

In [6]:
# Contar la cantidad de productos comprados por cada usuario
productos_user_counts = order_products.groupby('user_id').size().reset_index(name='product_count')

productos_user_counts['product_count'].describe()

count    206209.000000
mean        164.004025
std         206.014923
min           3.000000
25%          44.000000
50%          90.000000
75%         196.000000
max        3725.000000
Name: product_count, dtype: float64

In [7]:
# Calcular cuartiles
Q1 = productos_user_counts['product_count'].quantile(0.25)
Q3 = productos_user_counts['product_count'].quantile(0.75)

# Calcular el rango intercuartílico (IQR)
IQR = Q3 - Q1

# Definir los límites para detectar los outliers
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Filtrar los datos para eliminar los outliers
productos_user_counts_filtered = productos_user_counts[(productos_user_counts['product_count'] >= lower_limit) & (productos_user_counts['product_count'] <= upper_limit)]
productos_user_counts_filtered['product_count'].describe()

187868


Se filtra el dataframe principal para reducir la cantidad de registros.

In [8]:
# Obtener los conjuntos filtrados de productos y usuarios
filtered_product_ids = product_order_counts_filtered['product_id'].tolist()
filtered_user_ids = productos_user_counts_filtered['user_id'].tolist()

# Filtrar el dataframe original para incluir solo los que no son outliers
filtered_order_products = order_products[(order_products['product_id'].isin(filtered_product_ids)) & (order_products['user_id'].isin(filtered_user_ids))]

filtered_order_products.describe()

33819106
3073472


Se realiza un intento de elaborar una tabla con los usuarios y los productos que han comprado, especificando también su cantidad. No obstante, el kernel no pudo ejecutar la línea de código debido a su complejidad computacional: /var/folders/1l/48wfxxd93tx624h4f207pz180000gp/T/ipykernel_51289/2200060898.py:1: PerformanceWarning: The following operation may generate 7367292426 cells in the resulting pandas object. cross_order_product = pd.crosstab(filtered_order_products.user_id, filtered_order_products.product_id). Por lo tanto, no es posible realizar un agrupamiento de usuarios y producto específicos para este conjunto de datos. Sin embargo, se propone realizar el agrupamiento a nivel pasillo y departamento; con el propósito de identificar a los distintos grupos de usuarios.

In [26]:
cross_order_product = pd.crosstab(filtered_order_products.user_id, filtered_order_products.product_id)
cross_order_product.tail(20)

/var/folders/1l/48wfxxd93tx624h4f207pz180000gp/T/ipykernel_51289/2200060898.py:1: PerformanceWarning: The following operation may generate 7367292426 cells in the resulting pandas object.
  cross_order_product = pd.crosstab(filtered_order_products.user_id, filtered_order_products.product_id)


: 

In [ ]:
# Agrupar los usuarios junto con los pasillos y los departamentos para categorizarlos.

### 3.2 Market Basket Analysis with Association Rules